In [62]:
import fitsio as fio
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import time as time

In [63]:
mem = fio.FITS('/lsst/troxel/y1a1/lgt5_member_mcal_combined.fits')[-1] # Member/shape catalog
clus = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt5_desformat_catalog.fit')[-1] # Cluster catalog
cent = fio.FITS('/lsst/troxel/y1a1/central_member_shapes.fits')[-1] # Central/shape catalog

In [64]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3) # Use standard cosmological model

In [65]:
# Slice to the galaxies of specified catalog and cluster
def get_clus(catalog, clus_id):
    clus_mems = catalog.read()[np.in1d(catalog['MEM_MATCH_ID'].read(), clus_id, assume_unique=False)]
    
    return clus_mems

In [66]:
# Slice to the central galaxy of specified cluster
def get_cent(clus_id):
    clus_cent = get_clus(cent, clus_id)
    
    return clus_cent

In [67]:
# Slice to satellite galaxies of specified cluster
def get_sat(clus_id):
    mems = get_clus(mem, clus_id)
    cen = get_cent(clus_id)
    mask = np.in1d(mems, cen, assume_unique=True, invert=True)
    mems = mems[mask]
    
    return mems

In [68]:
# Calculate position angle
def pos_ang(e1, e2):
    alpha = np.degrees(np.arctan2(e2, e1)/2) * u.deg
    
    return alpha

In [69]:
# Calculate satellite angular position with respect to central
def sat_ang_pos(clus_id):
    sat_ra = get_sat(clus_id)['RA'] * u.deg
    sat_dec = get_sat(clus_id)['DEC'] * u.deg
    cent_ra = np.ones(len(sat_ra)) * get_cent(clus_id)['RA'] * u.deg
    cent_dec = np.ones(len(sat_dec)) * get_cent(clus_id)['DEC'] * u.deg
    sat_pos = SkyCoord(sat_ra, sat_dec, frame='icrs')
    cent_pos = SkyCoord(cent_ra, cent_dec, frame='icrs')
    sat_ang = cent_pos.position_angle(sat_pos).to(u.deg)
    
    return sat_ang

In [70]:
# Calculate central galaxy alignment angle
def cent_gal_ang(clus_id):
    sat_pos_ang = sat_ang_pos(clus_id)
    cent_pos_ang = np.ones(len(sat_pos_ang)) * pos_ang(get_cent(clus_id)['e1'], get_cent(clus_id)['e2'])
    theta = sat_pos_ang - cent_pos_ang
    theta = theta.to(u.rad)
    theta = np.arcsin(np.sin(theta))
    theta = np.arccos(np.cos(theta))
    theta = theta.to(u.deg)
    
    return theta

In [71]:
# Calculate central galaxy alignment angle for all viable clusters
def cent_ang_all():
    cent_id = cent['MEM_MATCH_ID'].read()
    new_array = np.zeros(len(mem.read())-len(cent.read()))
    k = 0
    for i in range(3000):
        cga = cent_gal_ang(cent_id[i])
        new_array[k:k+len(cga)] = cga
        k = k + len(cga)
    
    return new_array

In [72]:
# Count the satellite galaxies in each quadrant with specified axis angle (from horizontal)
def quad_counts(clus_id, theta):
    sat_pos_ang = sat_ang_pos(clus_id)
    theta = theta * u.deg
    q1_count = np.sum(np.logical_and(np.less_equal(theta, sat_pos_ang), np.less(sat_pos_ang, theta+90*u.deg)))
    q2_count = np.sum(np.logical_and(np.less_equal(theta+90*u.deg, sat_pos_ang), np.less(sat_pos_ang, theta+180*u.deg)))
    q3_count = np.sum(np.logical_and(np.less_equal(theta+180*u.deg, sat_pos_ang), np.less(sat_pos_ang, theta+270*u.deg)))
    q4_count = np.sum(np.less_equal(theta+270*u.deg, sat_pos_ang)) + np.sum(np.less(sat_pos_ang, theta))
    
    return np.array([q1_count, q2_count, q3_count, q4_count])

In [73]:
# Create array of quadrant counts for n angles between 0 and 90 degrees
def q_counts_array(clus_id, n):
    new_array = np.ones(n, dtype=[('angle',float)] + [('counts',float,(4,))] + [('chi_sq',float)])
    for i in range(n):
        new_array['angle'][i] *= 90 - i*90/n
        q_counts = quad_counts(clus_id, i*90/n)
        new_array['counts'][i] *= q_counts
        new_array['chi_sq'][i] *= stats.chisquare(q_counts)[0]
    chi2_sort = np.argsort(new_array['chi_sq'])
    new_array = new_array[chi2_sort]
    
    return new_array

In [74]:
# Calculate second moment M_ab
def mom_2(p, x, y, a, b):
    mom = np.sum(p*a*b/(x**2+y**2))/np.sum(p)
    
    return mom

In [75]:
# Calculate cluster position angle
def clus_pa(clus_id):
    sat_ra = get_sat(clus_id)['RA']
    sat_dec = get_sat(clus_id)['DEC']
    cent_ra = np.ones(len(sat_ra)) * get_cent(clus_id)['RA']
    cent_dec = np.ones(len(sat_dec)) * get_cent(clus_id)['DEC']
    p_i = get_sat(clus_id)['P'] * get_sat(clus_id)['PFREE']
    x_i = sat_ra - cent_ra
    y_i = sat_dec - cent_dec
    m_xx = mom_2(p_i, x_i, y_i, x_i, x_i)
    m_xy = mom_2(p_i, x_i, y_i, x_i, y_i)
    m_yy = mom_2(p_i, x_i, y_i, y_i, y_i)
    beta = np.degrees(np.arctan2(2*m_xy, (m_xx-m_yy))/2) * u.deg
    
    return beta

In [76]:
# Calculate difference between cluster position angle and central galaxy position angle
def alignment(clus_id):
    cent_pa = pos_ang(get_cent(clus_id)['e1'], get_cent(clus_id)['e2'])
    clust_pa = clus_pa(clus_id)
    theta = cent_pa - clust_pa
    theta = theta.to(u.rad)
    theta = np.arcsin(np.sin(theta))
    theta = np.arccos(np.cos(theta))
    theta = theta.to(u.deg)
    
    return theta

In [77]:
# Calculate alignment angle for all viable clusters
def alignment_array():
    cent_id = cent['MEM_MATCH_ID'].read()
    new_array = np.ones(len(cent_id))
    t1 = time.time()
    for i in range(len(cent_id)):
        new_array[i] = alignment(cent_id[i])[0].value
        t2 = time.time()
        print(t2-t1)

    return new_array    